<a href="https://colab.research.google.com/github/shashank-r97/Fraud-Project-3/blob/master/Style/Model_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
!pip install -U imbalanced-learn

     |████████████████████████████████| 163kB 38.8MB/s 
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [3]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases

Using TensorFlow backend.


In [4]:
%%time
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
df = df[df['attribute_name'] == 'style'].drop(columns = ['attribute_name'])
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'businesscasual' if x == 'business casual' else x)

CPU times: user 280 ms, sys: 53.3 ms, total: 334 ms
Wall time: 358 ms


In [5]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value
0,01DPGV4YRP3Z8J85DASGZ1Y99W,frame,les second medium noir,"minimal , modern styling meet refined luxury l...",accessory,casual
5,01DPH1DEN9G2WM7WAMJMD0A9W4,j crew,tie waist shirtdress stripe,take classic button silhouette turn ultra flat...,dressesandjumpsuits,casual
12,01E2KYW52BAG606GQ7A9H5R0KD,alo,interval microfleece pullover hoodie,articulate seam extra wide rib hem create shap...,unknown,casual
13,01DT513RRYT3SKH6X25G5VCH6B,chlo,leather ankle boot,heel measure approximately 55 mm 2 inch 30 mm ...,shoe boots ankle,androgynous
20,01E2KM0KW6NB1JKMZVRXR6H8G2,alo,stadium quarter zip hoodie,supersoft hoodie design elastic hem cuff perfe...,unknown,casual


In [0]:
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
X = df['text'].values
y = pd.get_dummies(df['attribute_value']).values

In [0]:
def encode_1gram(X, mode = 'binary'):
    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [0]:
def encode_2gram(X, mode='binary'):
    phrases = Phrases(X, min_count=30)
    bigrams = Phraser(phrases)
    X = list(bigrams[X])

    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [10]:
Tokenizer()

In [0]:
X1, length1, vocab_size1 = encode_1gram(X, mode = 'tfidf')

In [0]:
X2, length2, vocab_size2 = encode_2gram(X, mode = 'tfidf')

In [0]:
#X1 = X1.reshape(-1, 155, 1)
#X2 = X2.reshape(-1, 155, 1)

In [14]:
X1.shape

(10887, 155)

In [0]:
X = np.concatenate([X1, X2], axis = 1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [0]:
def define_model():
    inputs = Input(shape=(length1+length2,))
    #x1 = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(inputs1)
    embedding = Embedding(input_dim=vocab_size1, output_dim=100)(inputs)

    #inputs2 = Input(shape=(length2,))
    #x2 = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(inputs2)
    #dense2 = Dense(100, activation='relu')(inputs2)
    #embedding2 = Embedding(input_dim=vocab_size2, output_dim=100)(inputs2)

    #merged = concatenate([inputs1, inputs1])
    x = LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.15)(embedding)

    #x = Dense(100, activation = 'relu')(inputs)


    x = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(x)
    x = Conv1D(filters=num_classes, kernel_size=310, padding='valid')(x)
    x = Reshape((num_classes,))(x)
    #x = Dense(num_classes)(x)
    out = Activation('sigmoid')(x)

    model = Model(inputs = [inputs], outputs = out)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [32]:
model = define_model()
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 310)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 310, 100)          611300    
_________________________________________________________________
lstm_3 (LSTM)                (None, 310, 16)           7488      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 310, 11)           187       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1, 11)             37521     
_________________________________________________________________
reshape_3 (Reshape)          (None, 11)                0         
_________________________________________________________________
activation_3 (Activation)    (None, 11)                0   

In [0]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20944 samples, validate on 5236 samples
Epoch 1/10
20944/20944 [==============================] - 26s 1ms/step - loss: 0.3455 - accuracy: 0.8992 - val_loss: 0.4638 - val_accuracy: 0.9091
Epoch 2/10
20944/20944 [==============================] - 25s 1ms/step - loss: 0.2957 - accuracy: 0.9091 - val_loss: 0.4547 - val_accuracy: 0.9091
Epoch 3/10
20944/20944 [==============================] - 25s 1ms/step - loss: 0.2951 - accuracy: 0.9091 - val_loss: 0.4471 - val_accuracy: 0.9091
Epoch 4/10
20944/20944 [==============================] - 25s 1ms/step - loss: 0.2949 - accuracy: 0.9091 - val_loss: 0.4387 - val_accuracy: 0.9091
Epoch 5/10
20944/20944 [==============================] - 24s 1ms/step - loss: 0.2944 - accuracy: 0.9091 - val_loss: 0.4553 - val_accuracy: 0.9091
Epoch 6/10
20944/20944 [==============================] - 24s 1ms/step - loss: 0.2941 - accuracy: 0.9091 - val_loss: 0.4469 - val_accuracy: 0.9091
Epoch 7/10
20944/20944 [==============================] - 23s 1ms/ste

In [22]:
model.evaluate(X_test, y_test)

1089/1089 [==============================] - 0s 45us/step


[0.2962165831753264, 0.9068370461463928]

In [23]:
pd.Series(model.predict(X).argmax(axis = -1)).value_counts()

0    4019
3    1431
4    1237
5    1090
2     905
6     813
1     687
7     432
8     273
dtype: int64

In [24]:
model.predict(X)

array([[8.01080745e-03, 1.86787429e-03, 5.63683081e-03, ...,
        3.04006815e-01, 7.36444024e-04, 3.25369416e-03],
       [7.30718970e-02, 6.48583844e-03, 8.39708932e-03, ...,
        1.30199164e-01, 4.19933163e-03, 8.64354055e-03],
       [2.61472072e-02, 8.71820867e-01, 2.46914607e-02, ...,
        1.67083554e-02, 1.55212556e-03, 3.28178005e-03],
       ...,
       [9.33354422e-02, 2.34842338e-02, 1.53587028e-01, ...,
        1.42987460e-01, 1.19798165e-02, 2.36619152e-02],
       [9.33354422e-02, 2.34842338e-02, 1.53587028e-01, ...,
        1.42987460e-01, 1.19798165e-02, 2.36619152e-02],
       [9.33354422e-02, 2.34842338e-02, 1.53587028e-01, ...,
        1.42987460e-01, 1.19798165e-02, 2.36619152e-02]], dtype=float32)

In [25]:
pd.Series(y.argmax(axis = -1)).value_counts()

4     2633
8     1838
5     1488
3     1484
6      804
0      691
10     558
2      468
7      423
1      280
9      220
dtype: int64